In [7]:
import os
import json
import uuid
import yaml
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables.history import RunnableWithMessageHistory


In [8]:
with open("./api_keys.json", 'r') as f:
    api_keys = json.loads(f.read())

for key in api_keys.keys():
    os.environ[key] = api_keys[key]

In [9]:
gemini = ChatGoogleGenerativeAI(model="gemini-2.5-pro",temperature=0.2)
testing_version = "GoalInstructionOutputJava"

In [10]:
def read_prompts(path_to_file:str, tested_version:str):
    with open(path_to_file, "r") as f:
        prompt_data = yaml.safe_load(f)
    prompt_list=[]
    for task in prompt_data["tasks"]:
        for v in task["versions"]:
            if v["version"] == tested_version:
                prompt_list.append(v["text"])
    return prompt_list

In [11]:
chat_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

chain = chat_prompt | gemini #pipe style, replaces LLMChain()

message_history = InMemoryChatMessageHistory()
chain_with_memory = RunnableWithMessageHistory(
    chain,
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="history"
)

In [12]:
prompts = read_prompts("./prompts_tests.yaml", testing_version)
output_file_path = f"./test_results/test_conversation_version_{testing_version}_v0.md"
print("Prompts loaded")
markdown_output = [f"# Tests of prompts for Conversational Code Generation - version {testing_version}\n"]
with open(output_file_path, "w", encoding="utf-8") as f:
    f.write("\n".join(markdown_output))

print("Executing prompts and collecting responses")
for i, p in enumerate(prompts, start=1):
    print(f"Executing prompt {p}")
    result = chain_with_memory.invoke({"input": p},config={"configurable": {"session_id": str(uuid.uuid4())}})
    markdown_output = []
    markdown_output.append(f"## Prompt number {i}\n")
    markdown_output.append(f"**User:** {p}\n")
    markdown_output.append(f"**LLM Response:**\n\n{result.content}\n")
    with open(output_file_path, "a", encoding="utf-8") as f:
        f.write("\n".join(markdown_output))
print("Prompt execution finished")

print("Result saved to file")

Prompts loaded
Executing prompts and collecting responses
Executing prompt Goal: Setting up the role LLM should take for this conversation. 
Instruction:  I need to implement a new project from the beginning to end and I will need your help. Through this session, you should act as a senior java developer. I will divide my inputs into descriptions of the task, and requests for code generation. You should always carefully analyze the requirements and constrains. 
Expected Output: Confirmation of specified role. Apply it to every further part of the conversation.

Executing prompt Goal: Setting up a background for further conversation. 
Instruction: I have a new project on my hands. I need to prepare a clustering algorithm for quite a specific input data. 
My input consists of DNA sequences for different species, for which I need to calculate the similarity using Needleman-Wunsch algorithm and the predefined dictionary of swapping scores. Having the scores for all pairs I need to build ph